# Método de Ramificación y acotamiento 

In [1]:
from typing import List
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog, OptimizeResult
from dsplot.tree import BinaryTree
from typing import Tuple
from queue import Queue

In [2]:
def sub(number):
    subscript_map = {
        "0": "₀",
        "1": "₁",
        "2": "₂",
        "3": "₃",
        "4": "₄",
        "5": "₅",
        "6": "₆",
        "7": "₇",
        "8": "₈",
        "9": "₉",
        "i": "ᵢ",
        }

    return ''.join(subscript_map[digit] for digit in str(number))


# Clase que implementa los problemas de ramificación y acotamiento 
class Planteamiento:
    def __init__(self, funcion_objetivo: List, restricciones_desigualdad: List, restricciones_igualdad: List = [], 
                 modo = "max", tipo="Entero", variables_enteras: List= [], variables_continuas: List = []):
        if tipo not in ["Entero", "Mixto", "Binario"]:
            raise Exception(f"{modo} No es un tipo de problema valido.")
        self.modo = "Minimizar"
        self.tipo = tipo
        self.funcion_objetivo = np.array(funcion_objetivo)
        self.variables_enteras = variables_enteras
        self.variables_continuas = variables_continuas
        

        # El solver utilizado minimiza por default, si queremos maximizar hay que multiplicar la funcion objetivo por -1
        if modo == "max":
            self.modo = "Maximizar"
            self.funcion_objetivo *= -1
        
        self.restricciones_desigualdad = np.array([restriccion[:-1] for restriccion in restricciones_desigualdad])
        self.valor_restricciones_desigualdad = np.array([valor[-1] for valor in restricciones_desigualdad])
        
        if restricciones_igualdad:
            self.restricciones_igualdad = np.array([restriccion[:-1] for restriccion in restricciones_igualdad])
            self.valor_restricciones_igualdad = np.array([valor[-1] for valor in restricciones_igualdad])
        else:
            self.restricciones_igualdad = []
            self.valor_restricciones_igualdad = []
        
        if tipo == "Binario":
            numero_variables = len(self.funcion_objetivo)
            for i in range(numero_variables):
                temp = [0.0] * numero_variables
                temp[i] = 1.0
                self.restricciones_desigualdad = np.concatenate((self.restricciones_desigualdad, np.array([temp])), axis=0)
                self.valor_restricciones_desigualdad = np.concatenate((self.valor_restricciones_desigualdad, np.array([1.0])), axis=0)


    def __str__(self) -> str:
        modelo = ""
        
        # Formato a la función objetivo
        if self.modo == "Minimizar":
            modelo += "Minimizar\nz = "
            for i, x in enumerate(self.funcion_objetivo, start=1):
                modelo += f"{x}x{sub(i)}"
                try:
                    if funcion_objetivo[i] >= 0:
                        modelo += "+"
                except IndexError:
                    continue
        else:
            modelo += "Maximizar\nz = "
            for i, x in enumerate(self.funcion_objetivo, start=1):
                modelo += f"{-x}x{sub(i)}"
                try:
                    if funcion_objetivo[i] >= 0:
                        modelo += "+"
                except IndexError:
                    continue
        
        modelo += "\n\nSujeto a:\n\n"

        # Formato a las restricciones
        for j, restriccion in enumerate(self.restricciones_desigualdad):
            for i, x in enumerate(restriccion, start=1):
                modelo += f"{x}x{sub(i)}"
                try:
                    if restriccion[i] >= 0:
                        modelo += "+"
                except IndexError:
                    continue
            modelo += f" <= {self.valor_restricciones_desigualdad[j]}\n"
            
        if self.tipo == "Entero":
            modelo += f"x{sub('i')} ∈ ℤ"

        elif self.tipo == "Mixto":
            
            modelo += "x"
            for var in self.variables_continuas:
                modelo += f"{sub(var)},"
            modelo += "\b ∈ ℝ\n"

            modelo += "x"
            for var in self.variables_enteras:
                modelo += f"{sub(var)},"
            modelo += "\b ∈ ℤ"

        elif self.tipo == "Binario":
            modelo += f"x{sub('i')} ∈ " + "{0,1}"
        return modelo
    

    def solve(self) -> OptimizeResult:
        if self.restricciones_igualdad:
            return linprog(self.funcion_objetivo, A_ub=self.restricciones_desigualdad, b_ub=self.valor_restricciones_desigualdad,
                                     A_eq=self.restricciones_igualdad, b_eq=self.valor_restricciones_igualdad)
        else:
            return linprog(self.funcion_objetivo, A_ub=self.restricciones_desigualdad, b_ub=self.valor_restricciones_desigualdad)

In [3]:
funcion_objetivo = [3.0, 1.0, 3.0]

Restricciones = [[-1.0, 2.0, 1.0, 4.0],
                 [0.0, 4.0, -3.0, 2.0],
                 [1.0, -3.0, 2.0, 3.0]]

ejemplo =  Planteamiento(funcion_objetivo=funcion_objetivo, restricciones_desigualdad=Restricciones)
solucion = ejemplo.solve()
print(ejemplo)
# print(ejemplo.solucion)

Maximizar
z = 3.0x₁+1.0x₂+3.0x₃

Sujeto a:

-1.0x₁+2.0x₂+1.0x₃ <= 4.0
0.0x₁+4.0x₂-3.0x₃ <= 2.0
1.0x₁-3.0x₂+2.0x₃ <= 3.0
xᵢ ∈ ℤ


In [6]:
def is_approx_integer(float_number, tolerance=1e-6):
    absolute_difference = abs(float_number - round(float_number))
    return absolute_difference < tolerance



class Nodo:
    def __init__(self, modelo) -> None:
        self.right: Nodo or None = None
        self.left: Nodo or None = None
        self.problema: Planteamiento = modelo
        self.ramificado = False
        self.solucion: OptimizeResult = self.problema.solve()
        

    def __str__(self) -> str:
        resultado = f"Variables: \n"
        for index, variable in enumerate(self.problema.solucion.x, start=1):
            if is_approx_integer(variable, tolerance=1e-8):
                resultado += f"x{sub(f'{index}')} = {int(variable)}\n"
            else:
                resultado += f"x{sub(f'{index}')} = {variable}\n"
        if self.problema.modo == "Maximizar":
            resultado += f"\nZ: {abs(self.problema.solucion.fun)}"
        else:
            resultado += f"\nZ: {self.problema.solucion.fun}"
        return resultado
    
    def ramificar_binario(self):
        if self.solucion.

    
    def ramificar_entero(self):
        pass


    def ramificar_mixto(self):
        pass


class Arbol:
    def __init__(self, raiz: Nodo) -> None:
        self.raiz = raiz


    def validar_restricciones(self, nodo: Nodo) -> Tuple[bool, str]:
        if nodo.problema.tipo == "Entero":
            for variable in nodo.problema.solucion.x:
                if is_approx_integer(variable):
                    continue
                else:
                    return False, "aún hay reales en la solución"
            return True, "Solución factible"
        
    def recorrer(self):
        cola_nodos: Queue = Queue()
        orden_nodos = []
        if self.raiz is not None:
            orden_nodos.append(self.raiz)
            cola_nodos.put(self.raiz)
        while not cola_nodos.empty():
            nodo_actual: Nodo = cola_nodos.get()
            if nodo_actual.right is not None:
                orden_nodos.append(nodo_actual.right)
                cola_nodos.put(nodo_actual.right)
            else:
                orden_nodos.append(None)

            if nodo_actual.left is not None:
                cola_nodos.put(nodo_actual.left)
                orden_nodos.append(nodo_actual.left)
            else:
                orden_nodos.append(None)
        
        return orden_nodos


    def representar(self):
        nodos: List[Nodo] = self.recorrer()
        representacion = []
        for nodo in nodos:
            if nodo is not None:
                representacion.append(f"{nodo}")
            else:
                representacion.append("None")
        
        for item in representacion:
            print(f"{item}: {type(item)}")

        image_representation = BinaryTree(nodes=representacion)
        image_representation.plot(orientation="LR", shape="circle", fill_color="#346eeb")
        from PIL import Image
        img = Image.open(f"tree.png")
        img.show()
        


    def ramificar(self, nodo):
        valor = self.validar_restricciones(nodo)
        print(valor[1])

In [7]:
arbol_ejemplo = Arbol(Nodo(ejemplo))
arbol_ejemplo.representar()


# prueba_nodos = [f"{Nodo(ejemplo)}", "2", None]
# image_representation = BinaryTree(nodes=prueba_nodos)
# image_representation.plot(output_path=f"./prueba.png", orientation="LR", shape="circle", fill_color="#346eeb")
# from PIL import Image
# img = Image.open(f"prueba.png")
# img.show()

Variables: 
x₁ = 5.333333333333333
x₂ = 2
x₃ = 3.333333333333333

Z: 29.0: <class 'str'>
None: <class 'str'>
None: <class 'str'>


In [2]:
from utilities import Arbol, Nodo
funcion_objetivo = np.array([-3.0, -1.0, -3.0])

restricciones = {"desigualdades":np.array([[-1.0, 2.0, 1.0], [0.0, 4.0, -3.0], [1.0, -3.0, 2.0]]),
                            "objetivo_desigualdades": np.array([4.0, 2.0, 3.0]),
                            "igualdades": np.array([]),
                            "objetivo_igualdades": np.array([])}

coso = Arbol(Nodo(funcion_objetivo, restricciones, "Maximizar", "Entero"))
print(coso.recorrer_2())




[5.33333333 3.         3.33333333] aún tienen reales en la solución
[5.         2.85714286 3.28571429] aún tienen reales en la solución
Variables: 
x₁ = 5.333333333333333
x₂ = 2
x₃ = 3.333333333333333

Z: 29.0
└── Variables: 
x₁ = 5
x₂ = 2.857142857142857
x₃ = 3.2857142857142856

Z: 27.714285714285715
    └── Variables: 
x₁ = 5
x₂ = 2
x₃ = 2

Z: 23.0



In [2]:
funcion_objetivo = np.array([ -9.,  -5.,  -6.,  -4.])

restricciones = {'desigualdades': np.array([[ 6.,  3.,  5.,  2.],
       [ 0.,  0.,  1.,  1.],
       [-1.,  0.,  1.,  0.],
       [ 0., -1.,  0.,  1.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]]),
       'objetivo_desigualdades': np.array([10.,  1.,  0.,  0.,  1.,  1.,  1.,  1.]),
       'igualdades': np.array([]),
       'objetivo_igualdades': np.array([])}

print(linprog(c=funcion_objetivo, A_ub=restricciones["desigualdades"], b_ub=restricciones["objetivo_desigualdades"]))

from dsplot.tree import 

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -16.5
              x: [ 8.333e-01  1.000e+00  0.000e+00  1.000e+00]
            nit: 1
          lower:  residual: [ 8.333e-01  1.000e+00  0.000e+00  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  1.500e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  8.333e-01  0.000e+00
                              1.667e-01  0.000e+00  1.000e+00  0.000e+00]
                 marginals: [-1.500e+00 -0.000e+00 -0.000e+00 -0.000e+00
                             -0.000e+00 -5.000e-01 -0.000e+00 -1.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0
